# setup

In [43]:
import sys
sys.path.append('../../..')

In [44]:
from copulae.input import generate_copula_net_input
from copulae.training.cflax.bilogit import PositiveBiLogitCopula
from copulae.training.cflax.mono_aux import PositiveLayer
from copulae.training.cflax.mono_aux import EluPOne

from copulae.training.loss import sq_error, sq_error_partial, copula_likelihood
from copulae.training import setup_training

In [45]:
import jax.numpy as jnp
import jax.scipy.stats as jss
import jax

from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import optax
import pickle
from scipy.stats import bootstrap

In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# data
https://github.com/yutingng/gen-AC.git

In [47]:
import pandas as pd
import scipy
import copy

from sklearn.model_selection import train_test_split

In [48]:
def add_train_random_noise(data, num_adds):
  new_data = np.random.rand(num_adds, data.shape[1])
  return np.concatenate((data, new_data), axis = 0)

In [49]:
def rank_normalization(X):
  X = copy.deepcopy(X)
  for z in X:
      ndata = z.shape[0]
      gap = 1./(ndata+1)
      nfeats = z.shape[1]
      for i in range(nfeats):
          z[:, i] = scipy.stats.rankdata(z[:, i], 'ordinal')*gap
  return X

In [50]:
!git clone https://github.com/yutingng/gen-AC.git

fatal: destination path 'gen-AC' already exists and is not an empty directory.


In [51]:
class Boston():
  def __init__(self):
    # read
    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep = "\s+", skiprows = 22, header = None)
    X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
    y = raw_df.values[1::2, 2]

    # split
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, random_state = 142857)
    X_train = np.concatenate((X_train, y_train[:, None]), axis = 1)
    X_test  = np.concatenate((X_test, y_test[:, None]), axis = 1)

    # norm
    [X_train, X_test] = rank_normalization([X_train, X_test])

    # noise
    X_train = add_train_random_noise(X_train, int(X_train.shape[0]*0.01))

    # 2d
    train_data = X_train[:, [0, 13]]
    test_data = X_test[:, [0, 13]]

    # flip
    train_data[:, 0] = 1 - train_data[:, 0]
    test_data[:, 0] = 1 - test_data[:, 0]

    self.train_y = train_data[:, 1].reshape(-1, 1)
    self.train_x = train_data[:, 0].reshape(-1, 1)
    self.validation_y = test_data[:, 1].reshape(-1, 1)
    self.validation_x = test_data[:, 0].reshape(-1, 1)

In [52]:
class INTC_MSFT():
  def __init__(self):
    # read
    intel_f = open('gen-AC/data/raw/INTC_MSFT_GE/INTEL.data', 'r')
    intel = np.array(list(map(float, intel_f.readlines())))

    ms_f = open('gen-AC/data/raw/INTC_MSFT_GE/MS.data', 'r')
    ms = np.array(list(map(float, ms_f.readlines())))

    ge_f = open('gen-AC/data/raw/INTC_MSFT_GE/GE.data', 'r')
    ge = np.array(list(map(float, ge_f.readlines())))

    # split
    X = np.concatenate((intel[:, None], ms[:, None]), axis = 1)
    X_train, X_test, _, _ = train_test_split(X, X, shuffle = True, random_state = 142857)

    # norm
    [X_train, X_test] = rank_normalization([X_train, X_test])

    # 2d, noise
    train_data = X_train[:, [0, 1]]
    train_data = add_train_random_noise(train_data, int(train_data.shape[0]*0.01))
    test_data = X_test[:, [0, 1]]

    self.train_y = train_data[:, 1].reshape(-1, 1)
    self.train_x = train_data[:, 0].reshape(-1, 1)
    self.validation_y = test_data[:, 1].reshape(-1, 1)
    self.validation_x = test_data[:, 0].reshape(-1, 1)

In [53]:
class GOOG_FB():
  def __init__(self):
    # read
    goog_f = open('gen-AC/data/raw/FB_GOOG/goog/close.vals', 'r')
    goog = np.array(list(map(float, goog_f.readlines())))

    fb_f = open('gen-AC/data/raw/FB_GOOG/fb/close.vals', 'r')
    fb = np.array(list(map(float, fb_f.readlines())))

    # split
    X = np.concatenate((goog[:, None], fb[:, None]), axis = 1)
    X_train, X_test, _, _ = train_test_split(X, X, shuffle=True, random_state=142857)

    # norm
    [X_train, X_test] = rank_normalization([X_train, X_test])

    # 2d, noise
    train_data = X_train[:, [0, 1]]
    train_data = add_train_random_noise(train_data, int(train_data.shape[0]*0.01))
    test_data = X_test[:, [0, 1]]

    self.train_y = train_data[:, 1].reshape(-1, 1)
    self.train_x = train_data[:, 0].reshape(-1, 1)
    self.validation_y = test_data[:, 1].reshape(-1, 1)
    self.validation_x = test_data[:, 0].reshape(-1, 1)

In [54]:
def get_set(D_val, data_points):
  points = D_val
  points = jnp.expand_dims(points, axis=0)

  # PDF and CDF for X
  kde_x = jss.gaussian_kde(data_points[0], bw_method='silverman')
  density_x = kde_x.evaluate(points[0, 0, :])
  cumulative_x = jnp.array([kde_x.integrate_box_1d(-jnp.inf, p) for p in points[0, 0, :]])

  # PDF and CDF for Y
  kde_y = jss.gaussian_kde(D[1], bw_method='silverman')
  density_y = kde_y.evaluate(points[0, 1, :])
  cumulative_y = jnp.array([kde_y.integrate_box_1d(-jnp.inf, p) for p in points[0, 1, :]])

  I_pdf = density_x.T * density_y.T
  I_pdf = jnp.expand_dims(I_pdf, axis=0)
  cdf_xy = jnp.array((cumulative_x, cumulative_y))
  cdf_xy = jnp.expand_dims(cdf_xy, axis=0)

  del density_x
  del density_y
  del cumulative_x
  del cumulative_y

  return points, I_pdf, cdf_xy

In [55]:
np.random.seed(30091985)
key = jax.random.PRNGKey(30091985)

# params

In [56]:
layer_widths = [128, 64, 32, 16]
losses = [
    (0.01, sq_error),
    (0.5, sq_error_partial),
    (0.1, copula_likelihood),
]
lr = 2e-3
n_iter = 10000
label = '2catsnl'

In [57]:
def L_d(losses, params, state):
  loss = jnp.zeros((1,len(losses)), dtype=jnp.float32)
  for i, (w, loss_func) in enumerate(losses):
    loss = loss.at[0, i].set(w * loss_func(params, state))
  return loss

losses_eval = [
    (1.0, sq_error),
    (1.0, sq_error_partial),
    (1.0, copula_likelihood),
]

# boston

In [58]:
model = PositiveBiLogitCopula(
    PositiveLayer(layer_widths, EluPOne, EluPOne, EluPOne)
)

In [59]:
data_loader = Boston()
D = np.array([data_loader.train_x, data_loader.train_y])[:, :, 0]
TrainingTensors = generate_copula_net_input(
    D=D,
    bootstrap=False
)

In [60]:
nn_C, nn_dC, nn_c, cop_state, forward, grad = setup_training(
    model, TrainingTensors, losses
)

In [61]:
_, subkey = jax.random.split(key)
init_params = model.init(subkey, TrainingTensors.UV_batches[0])
del subkey

params = init_params
optimizer = optax.adam(lr)
opt_state = optimizer.init(params)

In [62]:
best = 1e6
for i in tqdm(range(n_iter)):
    grads, cop_state = grad(params, cop_state)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    if i % 10 == 0:
        loss = L_d(losses_eval, params, cop_state)
        if loss[0][-1] < best:
          best_params = params
          best_cop_state = cop_state
          best = loss[0][-1]
        print('Iter {}. Loss {}'.format(i, loss))

  0%|          | 1/10000 [00:43<120:40:56, 43.45s/it]

Iter 0. Loss [[0.35064277 0.11263812 1.7683982 ]]


  0%|          | 12/10000 [01:08<1:43:43,  1.60it/s]

Iter 10. Loss [[0.29436633 0.08281236 1.6504513 ]]


  0%|          | 22/10000 [01:10<24:02,  6.92it/s]

...

100%|█████████▉| 9982/10000 [21:38<00:02,  6.11it/s]

Iter 9980. Loss [[ 0.05353623  0.29531145 19.931568  ]]


100%|█████████▉| 9992/10000 [21:40<00:01,  6.21it/s]

Iter 9990. Loss [[ 0.05353148  0.29531145 19.931568  ]]


100%|██████████| 10000/10000 [21:41<00:00,  7.68it/s]


In [63]:
density_graph_points, I_pdf, cdf_xy = get_set(D, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))
np.mean(yhat)

0.7136317

In [64]:
D_val = np.array([data_loader.validation_x, data_loader.validation_y])[:, :, 0]
density_graph_points, I_pdf, cdf_xy = get_set(D_val, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))

L_d(losses_eval, best_params, best_cop_state), np.mean(yhat)

(Array([[0.19702114, 0.05269191, 0.11791787]], dtype=float32), 0.38270202)

In [65]:
res = bootstrap(yhat, np.mean)
res.standard_error, res.confidence_interval

(0.24623609,
 ConfidenceInterval(low=0.014748462718399372, high=1.046027047570956))

In [66]:
with open('drive/MyDrive/copulae/models/{}_boston.pkl'.format(label), 'wb') as fp:
    pickle.dump(best_params, fp)

# intc-msft

In [67]:
model = PositiveBiLogitCopula(
    PositiveLayer(layer_widths, EluPOne, EluPOne, EluPOne)
)

In [68]:
data_loader = INTC_MSFT()
D = np.array([data_loader.train_x, data_loader.train_y])[:, :, 0]
TrainingTensors = generate_copula_net_input(
    D=D,
    bootstrap=False
)

In [69]:
nn_C, nn_dC, nn_c, cop_state, forward, grad = setup_training(
    model, TrainingTensors, losses
)

In [70]:
_, subkey = jax.random.split(key)
init_params = model.init(subkey, TrainingTensors.UV_batches[0])
del subkey

params = init_params
optimizer = optax.adam(lr)
opt_state = optimizer.init(params)

In [71]:
best = 1e6
for i in tqdm(range(n_iter)):
    grads, cop_state = grad(params, cop_state)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    if i % 10 == 0:
        loss = L_d(losses_eval, params, cop_state)
        if loss[0][-1] < best:
          best_params = params
          best_cop_state = cop_state
          best = loss[0][-1]
        print('Iter {}. Loss {}'.format(i, loss))

  0%|          | 1/10000 [00:25<70:32:10, 25.40s/it]

Iter 0. Loss [[0.35345787 0.11756773 1.6619747 ]]


  0%|          | 12/10000 [00:52<1:24:15,  1.98it/s]

Iter 10. Loss [[0.3061803  0.08734921 1.5935664 ]]


  0%|          | 22/10000 [00:53<24:11,  6.88it/s]

...

100%|█████████▉| 9982/10000 [28:56<00:02,  6.94it/s]

Iter 9980. Loss [[0.28450626 0.00990751 0.21684353]]


100%|█████████▉| 9992/10000 [28:58<00:01,  6.21it/s]

Iter 9990. Loss [[0.2916601  0.0105438  0.21708618]]


100%|██████████| 10000/10000 [28:59<00:00,  5.75it/s]


In [72]:
density_graph_points, I_pdf, cdf_xy = get_set(D, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))
np.mean(yhat)

0.16720495

In [73]:
D_val = np.array([data_loader.validation_x, data_loader.validation_y])[:, :, 0]
density_graph_points, I_pdf, cdf_xy = get_set(D_val, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))

L_d(losses_eval, best_params, best_cop_state), np.mean(yhat)

(Array([[0.3344723 , 0.00571043, 0.0841653 ]], dtype=float32), 0.18417437)

In [74]:
res = bootstrap(yhat, np.mean)
res.standard_error, res.confidence_interval

(0.037233505,
 ConfidenceInterval(low=0.11373733052025314, high=0.2609860961502455))

In [75]:
with open('drive/MyDrive/copulae/models/{}_intcmsft.pkl'.format(label), 'wb') as fp:
    pickle.dump(best_params, fp)

# goog-fb

In [76]:
model = PositiveBiLogitCopula(
    PositiveLayer(layer_widths, EluPOne, EluPOne, EluPOne)
)

In [77]:
data_loader = GOOG_FB()
D = np.array([data_loader.train_x, data_loader.train_y])[:, :, 0]
TrainingTensors = generate_copula_net_input(
    D=D,
    bootstrap=False
)

In [78]:
nn_C, nn_dC, nn_c, cop_state, forward, grad = setup_training(
    model, TrainingTensors, losses
)

In [79]:
_, subkey = jax.random.split(key)
init_params = model.init(subkey, TrainingTensors.UV_batches[0])
del subkey

params = init_params
optimizer = optax.adam(lr)
opt_state = optimizer.init(params)

In [80]:
best = 1e6
for i in tqdm(range(n_iter)):
    grads, cop_state = grad(params, cop_state)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    if i % 10 == 0:
        loss = L_d(losses_eval, params, cop_state)
        if loss[0][-1] < best:
          best_params = params
          best_cop_state = cop_state
          best = loss[0][-1]
        print('Iter {}. Loss {}'.format(i, loss))

  0%|          | 1/10000 [00:25<70:32:27, 25.40s/it]

Iter 0. Loss [[0.2908933  0.10825062 1.1103379 ]]


  0%|          | 11/10000 [00:49<2:20:13,  1.19it/s]

Iter 10. Loss [[0.31879115 0.14460634 0.5909476 ]]


  0%|          | 22/10000 [00:51<24:35,  6.76it/s]

...

100%|█████████▉| 9982/10000 [28:31<00:02,  6.95it/s]

Iter 9980. Loss [[nan nan nan]]


100%|█████████▉| 9992/10000 [28:32<00:01,  7.09it/s]

Iter 9990. Loss [[nan nan nan]]


100%|██████████| 10000/10000 [28:34<00:00,  5.83it/s]


In [81]:
density_graph_points, I_pdf, cdf_xy = get_set(D, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))
np.mean(yhat)

-1.0573965

In [82]:
D_val = np.array([data_loader.validation_x, data_loader.validation_y])[:, :, 0]
density_graph_points, I_pdf, cdf_xy = get_set(D_val, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))

L_d(losses_eval, best_params, best_cop_state), np.mean(yhat)

(Array([[ 0.30965075,  0.08800514, -1.7028393 ]], dtype=float32), -1.1556219)

In [83]:
res = bootstrap(yhat, np.mean)
res.standard_error, res.confidence_interval

(0.034720197,
 ConfidenceInterval(low=-1.221460777675915, high=-1.084587106168581))

In [84]:
with open('drive/MyDrive/copulae/models/{}_googfb.pkl'.format(label), 'wb') as fp:
    pickle.dump(best_params, fp)